RISE-xai-techniques-Getting_Started-presoftmax_adversarial_attacks_v1

This notebook demonstrates adversarial attacks on gradient-based attribution methods
(e.g., Grad-CAM) that use pre-softmax outputs. It builds on repositories:
- https://github.com/mlerma54/adversarial-attacks-on-saliency-maps
- https://github.com/lisadunlap/xai-techniques

The goal is to show how to fool saliency maps produced by explainable AI techniques.


Clone the adversarial attacks repository for code and data

In [ ]:
!git clone https://github.com/mlerma54/adversarial-attacks-on-saliency-maps.git

Provide your system-specific paths below

In [ ]:
root = '/content/adversarial-attacks-on-saliency-maps/'
images_dir = root + 'images/'


Verify the repository and image directories exist

In [ ]:
import os

for d in [root, images_dir]:
  print(d)
  assert(os.path.isdir(d))

Set the original image and model to use

In [ ]:
orig_image = images_dir + 'dog-and-cat.jpg'
orig_image

In [ ]:
orig_model = 'vgg19'  # Alternative: 'resnet18'

Clone repository containing various XAI techniques implementations

In [ ]:
!git clone https://github.com/lisadunlap/xai-techniques.git

In [ ]:
%cd /content/xai-techniques

Install the techniques package

In [ ]:
!pip install -e .

In [ ]:
%pwd

In [ ]:
%cd techniques

Import required libraries for image processing, model manipulation, and visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import torch
from torchvision import models, datasets
from torch.autograd import Variable

from techniques.utils import get_displ_img

Note: If using Python 3.10+, a modification in 'gcam.py' might be required
Reference: https://github.com/skorch-dev/skorch/issues/895


Modifications to do on the files:

[https://github.com/skorch-dev/skorch/issues/895](https://github.com/skorch-dev/skorch/issues/895)
```
python3.10 -c "from collections.abc import Sequence"
```
/content/xai-techniques/techniques/Grad_CAM/gcam.py
```
# !/usr/bin/env python
# coding: utf-8
#
# Author:   Kazuto Nakashima
# URL:      http://kazuto1011.github.io
# Created:  2017-05-26

from collections import OrderedDict
from collections.abc import Sequence

```






Install additional dependencies needed for explanation methods

In [ ]:
!pip install lime

Import functions to generate different explanation heatmaps

In [ ]:
from techniques.generate_grounding import gen_grounding, gen_all_groundings
save_path='../cat_dog/'

Set image dimensions for model input

In [ ]:
width = height = 224

## Sample Image

Load and preprocess the sample image
TODO: Fix color channel issues if any arise in display

In [ ]:

displ_img = cv2.cvtColor(cv2.imread(orig_image), cv2.COLOR_BGR2RGB)
resized_image = cv2.resize(displ_img, (width, height))
plt.axis('off')
plt.imshow(resized_image)

## === Generate Mask/Heatmap Explanation Methods ===

### Grad-CAM explanations for dog class indices, both explicit and indexed targets

In [ ]:
#generate image wrt a specific class index
gcam_expl_dog = gen_grounding(resized_image,
                              'gcam',
                              orig_model,
                              target_index=1,
                              show=True,
                              save=True,
                              save_path=save_path+'dog',
                              device=0,
                              index=False)

In [ ]:
#generate image wrt a specific class index
gcam_expl_dog = gen_grounding(resized_image,
                              'gcam',
                              orig_model,
                              target_index=242,
                              show=True,
                              save=True,
                              save_path=save_path+'dog',
                              device=0,
                              index=True)

Grad-CAM explanation for cat class

In [ ]:
#generate image wrt a specific class index
gcam_expl_cat = gen_grounding(resized_image,
                              'gcam',
                              orig_model,
                              target_index=282,
                              show=True,
                              save=True,
                              save_path=save_path+'cat',
                              device=0,
                              index=True)

### LIME explanation for dog and cat classes

In [ ]:
lime_expl= gen_grounding(resized_image,
                         'lime',
                         orig_model,
                         target_index=1,
                         show=True,
                         save=True,
                         save_path=save_path+'dog',
                         device=0)

In [ ]:
lime_expl_cat= gen_grounding(resized_image,
                             'lime',
                             orig_model,
                             target_index=282,
                             show=True,
                             save=True,
                             save_path=save_path+'cat',
                             device=0,
                             index=True)

### RISE explanation for dog and cat classes

In [ ]:
rise_expl = gen_grounding(resized_image,
                          'rise',
                          orig_model,
                          target_index=1,
                          show=True,
                          save=True,
                          save_path=save_path+'dog',
                          device=0)

TODO: fix indexing for cat class in RISE explanations

In [ ]:

rise_expl_cat = gen_grounding(resized_image,
                              'rise',
                              orig_model,
                              target_index=2,
                              show=True,
                              save=True,
                              save_path=save_path+'cat',
                              device=0)

# === Generate Saliency Map Explanation Methods ===

### Integrated Gradients explanations for dog and cat classes

In [ ]:
from techniques.generate_grounding import gen_grounding, gen_all_groundings
ig_expl_dog = gen_grounding(resized_image,
                       'ig',
                       orig_model,
                       show=True,
                       save=True,
                       target_index=242,
                       save_path=save_path+'dog',
                       device=0,
                       index=True)

In [ ]:
ig_expl_dog = gen_grounding(resized_image,
                       'ig',
                       orig_model,
                       show=True,
                       save=True,
                       target_index=282,
                       save_path=save_path+'cat',
                       device=0,
                       index=True)

### Vanilla Backpropagation explanations

In [ ]:
#generate image wrt a specific class index
bp_expl_dog = gen_grounding(resized_image,
                            'bp',
                            orig_model,
                            target_index=242,
                            show=True,
                            save=True,
                            save_path=save_path+'dog',
                            device=0,
                            index=True)

In [ ]:
#generate image wrt a specific class index
bp_expl_dog = gen_grounding(resized_image,
                            'bp',
                            orig_model,
                            target_index=282,
                            show=True,
                            save=True,
                            save_path=save_path+'cat',
                            device=0,
                            index=True)

### Guided Backpropagation explanations

In [ ]:
#generate image wrt a specific class index
gbp_expl_dog = gen_grounding(resized_image,
                            'gbp',
                            orig_model,
                            target_index=242,
                            show=True,
                            save=True,
                            save_path=save_path+'dog',
                            device=0,
                            index=True)

In [ ]:
#generate image wrt a specific class index
gbp_expl_cat = gen_grounding(resized_image,
                            'gbp',
                            orig_model,
                            target_index=282,
                            show=True,
                            save=True,
                            save_path=save_path+'cat',
                            device=0,
                            index=True)

### Deconvolution explanations

In [ ]:
#generate image wrt a specific class index
deconv_expl_dog = gen_grounding(resized_image,
                            'deconv',
                            orig_model,
                            target_index=242,
                            show=True,
                            save=True,
                            save_path=save_path+'dog',
                            device=0,
                            index=True)

In [ ]:
#generate image wrt a specific class index
deconv_expl_cat = gen_grounding(resized_image,
                            'deconv',
                            orig_model,
                            target_index=282,
                            show=True,
                            save=True,
                            save_path=save_path+'cat',
                            device=0,
                            index=True)

## === Generate All Explanation Methods at Once ===

In [ ]:
#generate image wrt a specific class index
all_expl_dog = gen_all_groundings(resized_image,
                              orig_model,
                              target_index=1,
                              show=True,
                              save=True,
                              save_path='./cat_dog',
                              device=0,
                              index=False)